In [2]:
%reload_ext autoreload
%autoreload 2

from glicko2 import Rating, Glicko2

### Glicko-2

Glicko-2 represents the strength of each team as a probability distribution. Initially, each team is associated with a normal prior distribution. As match outcomes accumulate, teams are approximated to a posterior distribution. When the team wins its rating increases. This increase is influenced by two key factors: firstly, the difference in ratings (mu), resulting in a more substantial increase when a poor-rated team wins, and secondly, rating deviations (rd), which lead to rapid ratings adjustments when there is limited historical match data available for the opponents.

In [95]:
# initialize ratings
rating1 = Rating(mu=1500, rd=200)
rating2 = Rating(mu=1500, rd=200)

rating1

Rating(mu=1500, rd=200, volatility=0.06)

In [98]:
# "disable" home field advantage
neutral_filed = 0

# apriori draw probability, this value is calculated from the boosting model
apriori_draw_proba = 0.33

In [99]:
# calculate outcome probabilities
home_win_proba, tie_proba, away_win_proba = Glicko2().probabilities(rating1, rating2, neutral_filed, apriori_draw_proba)
home_win_proba, tie_proba, away_win_proba

(0.318590211976103, 0.36281957604779397, 0.318590211976103)

The probabilities of a home win and away win are equal.

In [101]:
# possible outcomes: 'H' represents a win for the home team, 'D' is a draw, 'A' is a win for away team
# recalculate ratings after a home team win.

new_rating1, new_rating2 = Glicko2().rate(rating1, rating2, neutral_filed, 'H', apriori_draw_proba)
new_rating1, new_rating2

(Rating(mu=1578.8017117324646, rd=180.07828241994562, volatility=0.059998510217965345),
 Rating(mu=1421.1982882675354, rd=180.07828241994562, volatility=0.059998510217965345))

After the win of the first team, the rating of the first team increased by 78.8 points.

In [102]:
# initialize ratings again, now the first team is the favorite
rating1 = Rating(mu=1700, rd=200)
rating2 = Rating(mu=1500, rd=200)

new_rating1, new_rating2 = Glicko2().rate(rating1, rating2, neutral_filed, 'H', apriori_draw_proba)
new_rating1, new_rating2

(Rating(mu=1748.5954852560465, rd=182.9374462139586, volatility=0.059995663932024136),
 Rating(mu=1451.4045147439535, rd=182.9374462139586, volatility=0.059995663932024136))

The rating of the first team increased by only 48.6 points because it was already the favorite, and the model does not significantly change ratings

In [103]:
# initialize ratings again, now the first team is the favorite and rating deviations are small,
# small rating deviation means that we are confident about given ratings
rating1 = Rating(mu=1700, rd=100)
rating2 = Rating(mu=1500, rd=100)

new_rating1, new_rating2 = Glicko2().rate(rating1, rating2, neutral_filed, 'H', apriori_draw_proba)
new_rating1, new_rating2

(Rating(mu=1714.4177679857044, rd=97.61896663544542, volatility=0.059994079892625896),
 Rating(mu=1485.5822320142956, rd=97.61896663544542, volatility=0.059994079892625896))

The rating of the first team increased by only 14.4 points because it was already the favorite and RD were small

### Home Field Advantage

In [106]:
# initialize ratings
rating1 = Rating(mu=1500, rd=200)
rating2 = Rating(mu=1500, rd=200)

rating1

Rating(mu=1500, rd=200, volatility=0.06)

In [110]:
# apriori draw probability
apriori_draw_proba = 0.30

# define home filed advantage
home_field_advantage = 30

In [111]:
home_win_proba, tie_proba, away_win_proba = Glicko2().probabilities(rating1, rating2, home_field_advantage, apriori_draw_proba)
home_win_proba, tie_proba, away_win_proba

(0.3886971098298659, 0.3493805802816519, 0.26192230988848214)

The probability of a home win is higher than that of an away team win because of the home field advantage.

In [112]:
new_rating1, new_rating2 = Glicko2().rate(rating1, rating2, home_field_advantage, 'H', apriori_draw_proba)
new_rating1, new_rating2

(Rating(mu=1569.024063217785, rd=180.3560206268279, volatility=0.059997147634433194),
 Rating(mu=1430.975936782215, rd=180.3560206268279, volatility=0.059997147634433194))

After the win of the home team, the rating of the home team increased by 69 points.

In [113]:
new_rating1, new_rating2 = Glicko2().rate(rating1, rating2, home_field_advantage, 'D', apriori_draw_proba)
new_rating1, new_rating2

(Rating(mu=1489.979092702602, rd=180.3559894421362, volatility=0.059992424579571234),
 Rating(mu=1510.020907297398, rd=180.3559894421362, volatility=0.059992424579571234))

After a draw, the rating of the home team decreased despite the fact that the ratings were equal before the game due to the loss of home field advantage.

In [53]:
new_rating1, new_rating2 = Glicko2().rate(rating1, rating2, home_field_advantage, 'A', apriori_draw_proba)
new_rating1, new_rating2

(Rating(mu=1410.8956759134726, rd=180.3580688921342, volatility=0.06000036465459351),
 Rating(mu=1589.1043240865274, rd=180.3580688921342, volatility=0.060000364654593484))

After the win of the away team, the rating of away team increased by 89 points.